<a href="https://colab.research.google.com/github/data602sps/assignments/blob/master/05_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data 620**
Seung Min Song, Krutika Patel<br>

02/11/2024




# **Assignment: Graph Visualization**

Graph Visualization

This week's assignment is to:

1. Load a graph dataset of your choosing from a text file or other source.  If you take a large network dataset from the web (such as from Stanford Large Network Dataset Collection), please feel free at this point to load just a small subset of the nodes and edges.
2. Create basic analysis on the graph, including the graph’s diameter, and at least one other metric of your choosing.  You may either code the functions by hand (to build your intuition and insight), or use functions in an existing package. 
3. Use a visualization tool of your choice (NetworkX, Gephi, Pyvis, etc.) to display information.
4. Please record a short video (~ 5 minutes), and submit a link to the video in advance of our meet-up.

# **Overview**

Network analysis is one of the basic techniques of big data analysis. It is a technique to understand data relationships through a network and find central data. At its most basic, it can analyze networks between people, but it can be used in infinite ways if you want to analyze relationships such as words, websites, and countries.

Among them, today we will create a network of actors who appeared in the same work through imdb movie-related data, and see if they are all connected to each other and who will be the most popular.

## **Data Source**

Downloaded the csv data table from the link below:
* https://www.kaggle.com/datasets/darinhawley/imdb-films-by-actor-for-10k-actors

In [2]:
import pandas as pd

# Load dataset

actors = pd.read_csv('https://raw.githubusercontent.com/seung-m1nsong/620/main/HW2/actorfilms.csv')

actors.head()


,Actor,ActorID,Film,Year,Votes,Rating,FilmID
0,Fred Astaire,nm0000001,Ghost Story,1981,7731,6.3,tt0082449
1,Fred Astaire,nm0000001,The Purple Taxi,1977,533,6.6,tt0076851
2,Fred Astaire,nm0000001,The Amazing Dobermans,1976,369,5.3,tt0074130
3,Fred Astaire,nm0000001,The Towering Inferno,1974,39888,7.0,tt0072308
4,Fred Astaire,nm0000001,Midas Run,1969,123,4.8,tt0064664


In [3]:
actors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191873 entries, 0 to 191872
Data columns (total 7 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Actor    191873 non-null  object 
 1   ActorID  191873 non-null  object 
 2   Film     191873 non-null  object 
 3   Year     191873 non-null  int64  
 4   Votes    191873 non-null  int64  
 5   Rating   191873 non-null  float64
 6   FilmID   191873 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 10.2+ MB


## **Data preprocessing**

To optimize network analysis, it's essential to select appropriate data. Initially, due to the size of the current dataset, we'll narrow our focus to the 2020 and 2021 data exclusively. Furthermore, we'll filter the data to include only records with a Rating of 7 or higher.

In [4]:
# Filter for the years 2020 and 2021 with ratings >= 7
actors_2020_2021 = actors[((actors['Year'] == 2020) | (actors['Year'] == 2021)) & (actors['Rating'] >= 7)].dropna(subset=['Year', 'Rating'])

# Display the filtered DataFrame
actors_2020_2021.head()

,Actor,ActorID,Film,Year,Votes,Rating,FilmID
4401,Kenneth Branagh,nm0000110,Tenet,2020,323041,7.5,tt6723592
4594,Steve Buscemi,nm0000114,The King of Staten Island,2020,42901,7.1,tt9686708
4703,Nicolas Cage,nm0000115,The Croods: A New Age,2020,22014,7.0,tt2850386
4803,Neve Campbell,nm0000117,Clouds,2020,6539,7.5,tt6473066
5100,Kevin Costner,nm0000126,Zack Snyder's Justice League,2021,242474,8.2,tt12361974


In [5]:
# Get information about the filtered DataFrame
actors_2020_2021.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 593 entries, 4401 to 191845
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Actor    593 non-null    object 
 1   ActorID  593 non-null    object 
 2   Film     593 non-null    object 
 3   Year     593 non-null    int64  
 4   Votes    593 non-null    int64  
 5   Rating   593 non-null    float64
 6   FilmID   593 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 37.1+ KB


Following this procedure, we will streamline our dataset to solely include the actor's name. Therefore, we will create a new dataframe containing only the title (Film) and the actor's name (Actor). 

In [6]:
actors_2020_2021 = actors_2020_2021.loc[:, actors_2020_2021.columns.intersection(['Actor','Film'])]

actors_2020_2021.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 593 entries, 4401 to 191845
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Actor   593 non-null    object
 1   Film    593 non-null    object
dtypes: object(2)
memory usage: 13.9+ KB


## **Network Analysis Dataset Transformation**

To prepare the dataset for network analysis, we need to modify its format. Presently, when a title features multiple performers, multiple records are generated. Utilizing the groupby function, we will aggregate the cast members for each title into a list. This approach ensures the elimination of duplicate titles in the dataset.

In [7]:
actors_2020_2021_join = actors_2020_2021.groupby(['Film'])['Actor'].apply(list).reset_index()
actors_2020_2021_join


,Film,Actor
0,A Sunburnt Christmas,[Sullivan Stapleton]
1,After Love,[Joanna Scanlan]
2,Angela's Christmas Wish,"[Jared Harris, Caitriona Balfe, Ruth Negga, Mo..."
3,Angrezi Medium,"[Kareena Kapoor, Irrfan Khan]"
4,Another Round,"[Maria Bonnevie, Thomas Bo Larsen, Mads Mikkel..."
...,...,...
103,Uncle Vanya,"[Roger Allam, Richard Armitage, Toby Jones, Ai..."
104,Wolfwalkers,"[Sean Bean, Maria Doyle Kennedy, Simon McBurne..."
105,Words on Bathroom Walls,"[Andy Garcia, Walton Goggins, Beth Grant, Moll..."
106,Yeh Ballet,[Julian Sands]



Next, the dataset requires transformation to facilitate its utilization in network analysis. The network analysis dataframe should include 'source', 'target', and 'weight' columns. Each row represents a combination of two actors from the same work, with one actor designated as the 'source' and the other as the 'target'. The 'weight' indicates the frequency of occurrence for each combination.

To achieve this, we will identify all possible combinations of two actors from the same work and calculate the frequency of each combination.

In [8]:
from itertools import combinations

actors_2020_2021_join['Actor'] = actors_2020_2021_join['Actor'].apply(lambda l: list(combinations(l, 2)))
actors_2020_2021_join = actors_2020_2021_join.explode('Actor').dropna()
actors_2020_2021_join

,Film,Actor
2,Angela's Christmas Wish,"(Jared Harris, Caitriona Balfe)"
2,Angela's Christmas Wish,"(Jared Harris, Ruth Negga)"
2,Angela's Christmas Wish,"(Jared Harris, Moe Dunford)"
2,Angela's Christmas Wish,"(Caitriona Balfe, Ruth Negga)"
2,Angela's Christmas Wish,"(Caitriona Balfe, Moe Dunford)"
...,...,...
107,Zack Snyder's Justice League,"(Ray Fisher, Ryan Zheng)"
107,Zack Snyder's Justice League,"(Ray Fisher, Brooke Ence)"
107,Zack Snyder's Justice League,"(Kiersey Clemons, Ryan Zheng)"
107,Zack Snyder's Justice League,"(Kiersey Clemons, Brooke Ence)"


In [9]:
nt = pd.DataFrame(actors_2020_2021_join['Actor'].to_list(), columns=['source','target'])
nt.head()

,source,target
0,Jared Harris,Caitriona Balfe
1,Jared Harris,Ruth Negga
2,Jared Harris,Moe Dunford
3,Caitriona Balfe,Ruth Negga
4,Caitriona Balfe,Moe Dunford


## **Run network analysis**

Once the dataset is prepared, we will proceed with network analysis by converting the dataset into a graph using NetworkX. This involves representing each actor as a node in the graph and each combination of actors from the same work as an edge. The weight of each edge will be determined by the frequency of occurrence of the actor combination.

In [10]:
nt['weight'] = nt.groupby(['source', 'target'])['source'].transform('size')
nt = nt.dropna()
nt

,source,target,weight
0,Jared Harris,Caitriona Balfe,1
1,Jared Harris,Ruth Negga,1
2,Jared Harris,Moe Dunford,1
3,Caitriona Balfe,Ruth Negga,1
4,Caitriona Balfe,Moe Dunford,1
...,...,...,...
2864,Ray Fisher,Ryan Zheng,1
2865,Ray Fisher,Brooke Ence,1
2866,Kiersey Clemons,Ryan Zheng,1
2867,Kiersey Clemons,Brooke Ence,1


The number of connections an actor has (n) is calculated by aggregating the weight of each connection of an actor. The value is used to determine the connectedness of an actor.

In [11]:
node_sizes = nt.groupby('source').weight.agg(sum)
n = node_sizes.add_suffix('').reset_index()

n


,source,weight
0,Aaron Jeffery,1
1,Aaron Taylor-Johnson,6
2,Adam Brody,16
3,Adam Rothenberg,3
4,Agusta Eva Erlendsdottir,9
...,...,...
453,Zachary Levi,2
454,Zack Snyder,12
455,Zazie Beetz,1
456,Zeljko Ivanek,6


The connectedness of an actor is seperated into categories of up to 5, 10, 15, and greater. The seperation between the categories is shown by the color of the node. With the color scale going as the following: yellow, green, blue, purple.

In [12]:
node_sizes = nt.groupby('source').weight.agg(sum)
n = node_sizes.add_suffix('').reset_index()

In [13]:
import networkx as nx

G = nx.from_pandas_edgelist(nt, 'source', 'target',
                            create_using=nx.DiGraph(), edge_attr='weight')
#seperate nodes into categories
node_lst_5 = n['source'][n['weight']<6]
node_lst_5 = node_lst_5.values.tolist()
node_lst_10 = n['source'][(n['weight']>=6) & (n['weight']<=10)]
node_lst_10 = node_lst_10.values.tolist()
node_lst_15 = n['source'][(n['weight']>=11) & (n['weight']<=15)]
node_lst_15 = node_lst_15.values.tolist()

In [14]:
from pyvis.network import Network

net = Network(notebook=True)

#add nodes
for n in G:
    #set color based on category
    if n in node_lst_5:  
        color = 'khaki'
    elif n in node_lst_10:
        color = 'turquoise'
    elif n in node_lst_15:
        color = 'lightskyblue'
    else:
        color = 'orchid'
    net.add_node(n, label=n, color=color)

#add edges
net.add_edges(G.edges())

## **Visualization**

Visualize the network and save it as a file. The visualization results are displayed within the Jupyter notebook and saved as image files.



Looking at the graph we are able to visualize the connections and popularity of actors. The graphs allows us infer the connection degree of an actor via the color intensity of their node, with the darkest nodes representing the more popular and well connected actors. When looking at the influential figures at the center of the network, prolific supporting actors like Lil Rel Howery stand out.

In [15]:
# Show the network
net.show_buttons(filter_=['nodes'])

net.show('plot1.html')

plot1.html
